<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/genomic_benchmarks/blob/main/notebooks/How_To_Train_CNN_Classifier_With_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How To Train CNN Classifier With TF

This notebook demonstrates how to use `genomic_benchmarks` to train a neural network classifier on one of its benchmark datasets [human_nontata_promoters](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/tree/main/docs/human_nontata_promoters).

In [1]:
pip install genomic-benchmarks

     |████████████████████████████████| 2.3 MB 3.9 MB/s 
     |████████████████████████████████| 596 kB 37.6 MB/s 
     |████████████████████████████████| 271 kB 46.8 MB/s 
     |████████████████████████████████| 160 kB 52.8 MB/s 
  Created wheel for genomic-benchmarks: filename=genomic_benchmarks-0.0.3-py3-none-any.whl size=17311 sha256=14d60947c73f35752bc3f3201d867883a6ba693355aaf0d9510c1a8ce53c958f
  Stored in directory: /root/.cache/pip/wheels/34/4f/f6/401d087f70557375a949ecbd3cf43e9cf9dcfc971ea77c12db
Successfully built genomic-benchmarks
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Data download

With the function `download_dataset` downloads, we can download full-sequence form of the benchmark, splitted into train and test sets, one folder for each class.

In [2]:
from pathlib import Path
import tensorflow as tf

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Done.
Unzipping...Done.


In [3]:
info('human_nontata_promoters', 0)

Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


## TF Dataset object

To train the model with TensorFlow, we must create a TF Dataset. Because the directory structure of our benchmarks is ready for training, we can just call `tf.keras.preprocessing.text_dataset_from_directory` function as follows.

In [4]:
BATCH_SIZE = 64
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters'
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


## Text vectorization

To convert the strings to tensors, we internally use TF `TextVectorization` layer and splitting to characters.

In [5]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

['', '[UNK]', 'g', 'c', 't', 'a']

In [6]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

To get a baseline (other models can be compared to) we ship a package with [a simple CNN model](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/blob/main/src/genomic_benchmarks/models/tf.py). We have vectorized the dataset before training the model to speed up the process.

In [7]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [8]:
EPOCHS = 10

history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
424/424 [==============================] - 42s 24ms/step - loss: 0.4407 - binary_accuracy: 0.7983
Epoch 2/10
424/424 [==============================] - 10s 23ms/step - loss: 0.3850 - binary_accuracy: 0.8267
Epoch 3/10
424/424 [==============================] - 10s 23ms/step - loss: 0.3599 - binary_accuracy: 0.8401
Epoch 4/10
424/424 [==============================] - 10s 24ms/step - loss: 0.3439 - binary_accuracy: 0.8492
Epoch 5/10
424/424 [==============================] - 10s 23ms/step - loss: 0.3278 - binary_accuracy: 0.8571
Epoch 6/10
424/424 [==============================] - 10s 23ms/step - loss: 0.3128 - binary_accuracy: 0.8652
Epoch 7/10
424/424 [==============================] - 10s 23ms/step - loss: 0.2975 - binary_accuracy: 0.8721
Epoch 8/10
424/424 [==============================] - 10s 23ms/step - loss: 0.2879 - binary_accuracy: 0.8770
Epoch 9/10
424/424 [==============================] - 10s 23ms/step - loss: 0.2797 - binary_accuracy: 0.8808
Epoch 10/10
424/424

## Evaluation on the test set

Finally, we can do the same pre-processing for the test set and evaluate the accuracy of our model.

In [9]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 9034 files belonging to 2 classes.


In [10]:
model.evaluate(test_ds)

142/142 [==============================] - 3s 20ms/step - loss: 0.3356 - binary_accuracy: 0.8568


[0.3355793058872223, 0.8567633628845215]